In [61]:
#imports
import json
import Levenshtein as lev
import datetime
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
import nltk
import re
from collections import Counter
import json
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import regex
import string


In [33]:
def parse_file(filename):
    tweets_data = json.load(open(filename))
    
    stop_words = list(stopwords.words('english')) 
    
    original_tweets = []
    
    for tweet in tweets_data:
        if 'RT' not in tweet['text']:
            original_tweets.append(tweet)
    
    gg_stop_words = ['Globe', 'RT', 'http', 'Golden', 'Globes', 'GoldenGlobes', 'Goldenglobes', 'Goldenglobe', 'gg','golden globes', 'golden globe', 'goldenglobe','goldenglobes','gg2015','gg15','goldenglobe2015','goldenglobe15','goldenglobes2015','goldenglobes15', 'gg2013','gg13','goldenglobe2013','goldenglobe13','goldenglobes2013','goldenglobes13', 'rt', '2013', '2015']

    tokenizer = RegexpTokenizer(r'\w+')
    
    word_list = []
    
    for tweet in original_tweets:
        text = tweet['text']  
        time = tweet['timestamp_ms']
        words = tokenizer.tokenize(text)
        tweetText = []
        for w in words:
            if w not in stop_words and w not in gg_stop_words:
                tweetText.append(w)
        word_list.append((tweetText, time))
        
    return word_list

In [34]:
filename = "gg2013.json"
data = parse_file(filename)
data

[(['JLo', 'dress', 'eredcarpet'], 1358124338000),
 (['What',
   'making',
   'Sofia',
   'Vergara',
   'boobs',
   'stay',
   'like',
   'Magic',
   'Witchcraft'],
  1358124338000),
 (['Anne', 'Hathaway', 'got', 'living'], 1358124338000),
 (['Jennifer', 'Lopez', 'lace', 'dress', 'Thoughts'], 1358124338000),
 (['Podrán',
   'criticar',
   'Adele',
   'de',
   'su',
   'moda',
   'su',
   'maniquere',
   'pero',
   'jamás',
   'podrán',
   'poner',
   'en',
   'tela',
   'de',
   'juicio',
   'su',
   'TALENTO',
   'eonline',
   'RedCarpet'],
  1358124338000),
 (['US', 'Weakly'], 1358124338000),
 (['Hugh', 'Jackman', 'awesome'], 1358124338000),
 (['hellen', 'mirren', 'O'], 1358124338000),
 (['Jennifer', 'Lopez', 'dress', 'jaw', 'droppingly', 'amazing', 'redcarpet'],
  1358124338000),
 (['Jessica', 'Alba', 'absolutely', 'flawless', 'omg', 'redcarpet'],
  1358124338000),
 (['Werk', 'kerrywashington', 'Stunning', 'red', 'carpet'], 1358124338000),
 (['Sofia', 'Vergara', 'este', 'bueno', 'ell

In [35]:
import json
import re
tweets = json.load(open("gg2013answers.json"))
tweets['award_data']

{'best screenplay - motion picture': {'nominees': ['zero dark thirty',
   'lincoln',
   'silver linings playbook',
   'argo'],
  'presenters': ['robert pattinson', 'amanda seyfried'],
  'winner': 'django unchained'},
 'best director - motion picture': {'nominees': ['kathryn bigelow',
   'ang lee',
   'steven spielberg',
   'quentin tarantino'],
  'presenters': ['halle berry'],
  'winner': 'ben affleck'},
 'best performance by an actress in a television series - comedy or musical': {'nominees': ['zooey deschanel',
   'tina fey',
   'julia louis-dreyfus',
   'amy poehler'],
  'presenters': ['aziz ansari', 'jason bateman'],
  'winner': 'lena dunham'},
 'best foreign language film': {'nominees': ['the intouchables',
   'kon tiki',
   'a royal affair',
   'rust and bone'],
  'presenters': ['arnold schwarzenegger', 'sylvester stallone'],
  'winner': 'amour'},
 'best performance by an actor in a supporting role in a motion picture': {'nominees': ['alan arkin',
   'leonardo dicaprio',
   'phil

In [45]:
def get_winner(movie):
    answers = json.load(open("gg2013answers.json"))
    return answers["award_data"][movie]["winner"]

In [95]:
import Levenshtein as lev
def leven(str1, str2):
    Distance = lev.distance(str1.lower(), str2.lower())
    Ratio = lev.ratio(str1.lower(), str2.lower())
    if Ratio > .9:
        return True
    else:
        return False

In [104]:
import imdb
def isMovie(movie):
    ia = imdb.IMDb()
    movies = ia.search_movie(movie)
    if len(movies)==0:
        return False
    
    str1 = movie
    str2 = movies[0]['title']
    for m in movies:
        str2 = m['title']
        if leven(str1,str2):
            return str2
    return False
isMovie('lincoln')

'Lincoln'

In [105]:
import json
import re
tweets = json.load(open("gg2013.json"))
reg = regex.Regex()
# print(reg.film_award)
results={}
for movie in reg.award_names:
#     print(movie)
    if movie in reg.film_award:
#         movie = ' best original song - motion picture '
        search_term = get_winner(movie)
        # search_term = reg.getRegex(movie)
        
        word_size = 2
#         print(search_term)
        result = []
        for tweet in tweets:
            text = tweet['text'].lower()
            if 'RT' not in tweet['text']:
                if re.search(search_term, text):
                    text = text.replace(',', "")
                    text = text.replace('.', "")
                    text = text.replace('!', "")
                    #text.translate(str.maketrans('', '', string.punctuation))
                    #                     print(text)
                    A = re.findall(r'“(.*?)”', text)
                    B = re.findall(r'"(.*?)"', text)
                        
                    if len(A)!=0:
                        for sentence in A:
                            if len(sentence.split())<10:
                                if search_term not in sentence and '@' not in sentence and '#' not in sentence and 'best' not in sentence and sentence != '':
                                    movie_title = isMovie(sentence)
                                    if movie_title:
                                        result.append(movie_title)
                    if len(B)!=0:
                        for sentence in B:
                            if len(sentence.split())<10:
                                if search_term not in sentence and '@' not in sentence and '#' not in sentence and 'best' not in sentence and sentence != '':
                                    movie_title = isMovie(sentence)
                                    if movie_title:
                                        result.append(movie_title)
        
         
        if len(result)==0:
            for tweet in tweets:
                text = tweet['text']
                if 'RT' not in tweet['text']:
                    if re.search(search_term, text):
                            C = re.findall(r'-(.*?)-', text)
                            if len(C)!=0:
#                                 print(text)
                                for sentence in C:
                                    if len(sentence.split())<5:
                                        if search_term not in sentence and '@' not in sentence and '#' not in sentence:
                                            movie_title = isMovie(sentence)
                                            if movie_title:
                                                result.append(movie_title)
        print(movie)    
        print(Counter(result).most_common())  
        print()

        
        mystr = Counter(result).most_common()[0][0]
        if mystr == 'Comedy or Musical':
            mystr = Counter(result).most_common()[0][1]
            
        results[movie] = mystr
# for key in results.keys():
#     print(key,results[key])
# Counter(results).most_common()

best motion picture - drama
[('Lincoln', 17), ('Zero Dark Thirty', 6), ('Les Misérables', 3), ('Daredevil', 2), ('Life of Pi', 2), ('Gigli', 2), ('Django Unchained', 2), ('Maya', 1), ('Golden Globes', 1), ('Dream On', 1), ('Django', 1), ('Silver Linings Playbook', 1), ('Agradecimiento', 1), ('Maybe We Made a Mistake', 1), ('Academy', 1), ('Twitter Fail', 1), ('Girls', 1), ('Hype', 1), ('Les Miserables', 1), ('Homeland', 1), ('Game Change', 1), ('The Town', 1), ('Snubbed', 1)]

best motion picture - comedy or musical
[('Argo', 4), ('Skyfall', 1), ('That', 1), ('Django', 1), ('Silver Linings Playbook', 1), ('Anne Hathaway', 1), ('Bles', 1), ('Daddy', 1), ('Robo', 1)]

best animated feature film
[('Valiente', 1), ('Wreck-It Ralph', 1), ('Chosen', 1), ('Courage', 1), ('Girls', 1), ('Sorpresa', 1), ('Argo', 1)]

best foreign language film
[('Old Hollywood', 1), ('Clinton', 1), ('A Royal Affair', 1), ('Funny Games', 1)]

best screenplay - motion picture
[('Lincoln', 6), ('Argo', 2), ('D', 1)

In [106]:
import json
import re
tweets = json.load(open("gg2013.json"))
reg = regex.Regex()
# print(reg.film_award)
results={}
for movie in reg.award_names:
#     print(movie)
    if movie in reg.people_award:
#         movie = ' best original song - motion picture '
        search_term = get_winner(movie)
        # search_term = reg.getRegex(movie)
        
        word_size = 2
#         print(search_term)
        result = []
        for tweet in tweets:
            text = tweet['text'].lower()
            if 'RT' not in tweet['text']:
                if re.search(search_term, text):
                    text = text.replace(',', "")
                    text = text.replace('.', "")
                    text = text.replace('!', "")
                    #text.translate(str.maketrans('', '', string.punctuation))
                    #                     print(text)
                    A = re.findall(r'“(.*?)”', text)
                    B = re.findall(r'"(.*?)"', text)
                        
                    if len(A)!=0:
                        for sentence in A:
                            if len(sentence.split())<10:
                                if search_term not in sentence and '@' not in sentence and '#' not in sentence and 'best' not in sentence and sentence != '':
                                    result.append(sentence)
                    if len(B)!=0:
                        for sentence in B:
                            if len(sentence.split())<10:
                                if search_term not in sentence and '@' not in sentence and '#' not in sentence and 'best' not in sentence and sentence != '':
                                    result.append(sentence)

        
         
        if len(result)==0:
            for tweet in tweets:
                text = tweet['text']
                if 'RT' not in tweet['text']:
                    if re.search(search_term, text):
                            C = re.findall(r'-(.*?)-', text)
                            if len(C)!=0:
#                                 print(text)
                                for sentence in C:
                                    if len(sentence.split())<5:
                                        if search_term not in sentence and '@' not in sentence and '#' not in sentence:
                                            result.append(sentence)
        print(movie)    
        print(Counter(result).most_common())  
        print()

        
        mystr = Counter(result).most_common()[0][0]
        if mystr == 'Comedy or Musical':
            mystr = Counter(result).most_common()[0][1]
            
        results[movie] = mystr
# for key in results.keys():
#     print(key,results[key])
# Counter(results).most_common()

cecil b. demille award
[('coming out', 78), ('normal is not something to aspire', 6), ('tonight i feel like the prom queen', 3), ('came out', 3), ('candleshoe', 2), ('partner', 2), ("i'm 50", 2), ('privacy', 2), ('i am not honey boo boo child', 2), ('ame as pessoas e fique ao lado delas', 2), ('love people and stay beside them', 2), ('out', 2), ("here's to the next 50", 2), ('i wanna be seen to be understood deeply', 2), ('lost', 2), ('retro', 1), ('lifetime achievement award', 1), ("there's kristen stewarts dad", 1), ('date?', 1), ('nos gostamos de você', 1), ('quando as metralhadoras cospem', 1), ('mel gibson?', 1), ("don't panic bite the hamsters vamp vamp", 1), ('um…what?', 1), ('that', 1), ("jodie's golden globes", 1), ('role', 1), (' la habitacion del panico', 1), ('it is something to get away from', 1), ('home for the holidays', 1), ('are you nell? from the movie nell?', 1), ('contact', 1), ('be the hero of your own life story', 1), ('tonight i feel like a prom queen', 1), ('i a

best performance by an actor in a supporting role in a motion picture
[('django unchained', 23), ('homeland', 9), ('wir sind golden globes', 1), ('supporting', 1), ('accolade', 1), ('django desencadenado', 1), ('n-bomb', 1), ("you're secretly kinda handsome", 1), ('aficionado', 1), ('the north star is that one', 1), ('ta-dah', 1), ('maybe he his', 1), ("there aren't a lot of german jews", 1), ('for most boring acceptance speech', 1), ('parole chicago', 1), ('christopher waltz', 1), ("they're not reality", 1), ('besides awards is there anything you collect?', 1), ('game change', 1)]

best director - motion picture
[('argo', 58), ('argo fuck yourself', 3), ('gigli', 3), ("you're my everything", 3), ('girls', 3), ("i'd tell you but i'd have to kill you", 2), ('take that oscars', 2), ('mejor director', 2), ('daredevil', 2), ('bennifer', 2), ('the town', 2), ("it's like looking in a mirror", 1), ('beside eachother', 1), ('slicker than a 6 year old', 1), ('argo fuck yourselves oscar', 1), ('